In [146]:
%config IPCompleter.greedy=True

In [147]:
import collections
from datetime import date
from dateutil.relativedelta import *
from faker import Faker
from collections import namedtuple, Counter
from time import perf_counter
from functools import wraps
from random import shuffle, seed, randint, uniform, choice, sample
from faker.providers.company import Provider

In [148]:
Faker.seed(1)
fake = Faker()

In [149]:
def get_fake_profiles(type: str, count: int):
    '''
	it has two parameter:
	   1) type - dict , namedtuple
	   2) count - no. of fake profiles
	In this function we will create a count number of fake profile with there respective type
    '''
    fake_profiles = []
    if type == 'namedtuple':
        profile = namedtuple('Profile', ['job', 'company', 'ssn', 'residence', 'current_location',
                                         'blood_group', 'website', 'username', 'name', 'sex', 'address', 'mail', 'birthdate'])
        for i in range(count):
            fake_profiles.append(profile(**fake.profile()))
    elif type == 'dict':
        for i in range(count):
            fake_profiles.append(fake.profile())
    return fake_profiles


def get_largest_blood_group(collection: list, type: str):
    '''
	this funtion also have two parameter:
	   1) collection - collection of fake profiles
	   2) type - dict , namedtuple
	this basically return the most common blood group with count
    '''
    bg_counts = None
    if type == 'namedtuple':
        bg_counts = Counter(p.blood_group for p in collection)
    elif type == 'dict':
        bg_counts = Counter(p['blood_group'] for p in collection)
    return bg_counts.most_common()[0]


def get_oldest_birthdate(collection: list, type: str):
    '''
	this funtion also have two parameter:
	   1) collection - collection of fake profiles
	   2) type - dict , namedtuple
	this basically return the oldest birthday amoung the given collection
    '''
    oldest_birthdate = None
    if type == 'namedtuple':
        oldest_birthdate = min(
            collection, key=lambda a: a.birthdate).birthdate
    elif type == 'dict':
        oldest_birthdate = min(
            collection, key=lambda a: a['birthdate'])['birthdate']
    return oldest_birthdate


def get_average_age(collection: list, type: str):
    '''
	this funtion also have two parameter:
	   1) collection - collection of fake profiles
	   2) type - dict , namedtuple
	this basically return the average age amoung the given collection
    '''

    if type == 'namedtuple':
        return sum([relativedelta(date.today(), a.birthdate).years for a in collection])/len(collection)
    elif type == 'dict':
        return sum([relativedelta(date.today(), a['birthdate']).years for a in collection])/len(collection)

In [150]:
NO_OF_PROFILES = 10000
fake_profiles = get_fake_profiles('namedtuple', NO_OF_PROFILES)
fake_profiles_dict = get_fake_profiles('dict', NO_OF_PROFILES)

In [151]:
def get_average_time(n, func, *args, **kwargs):
    '''
    this basically return the running time of the given function upto n times
    '''
    start = perf_counter()
    for _ in range(n):
        value = func(*args, **kwargs)
    avg_run_time = (perf_counter() - start)/n
    return avg_run_time


N = 1000
for f in [get_largest_blood_group, get_oldest_birthdate, get_average_age]:
    print('testing:', f.__name__)
    t1 = get_average_time(N, f, fake_profiles, 'namedtuple')
    t2 = get_average_time(N, f, fake_profiles_dict, 'dict')

    if t1 < t2:
        print('dict is faster')
    else:
        print('named tuple is faster')

testing: get_largest_blood_group
dict is faster
testing: get_oldest_birthdate
dict is faster
testing: get_average_age
named tuple is faster


In [152]:
def get_company_stock(volatility,direction): 
    '''
    this function basically create fake company stocks
    '''  
    name = fake.unique.company()
    symbol = name[0:3].upper()+name[-3:].upper()
    open = round( uniform(10,1000),2)
    if direction == '-ve':
        volatility *= -1
    close =  round(open + (open * volatility * 0.2),2)
    high = close = round(open + (open * volatility * 0.5),2)
    low = close = round(open - (open * volatility * 0.5),2)
    return name, symbol, open, low,high, close

In [153]:
weights = sample(range(1,200),100)

In [154]:
stock_list = [company_stck(*get_company_stock(volatility = uniform(.02, .2),direction=choice(['+ve','-ve'])),w) for w in weights]


In [155]:
stock_list

[Company(name='Weaver Ltd', symbol='WEALTD', open=632.19, low=607.4, high=656.98, close=607.4, weight=88),
 Company(name='James, Graves and Phillips', symbol='JAMIPS', open=532.65, low=505.07, high=560.23, close=505.07, weight=35),
 Company(name='Johnson Group', symbol='JOHOUP', open=152.75, low=155.52, high=149.98, close=155.52, weight=191),
 Company(name='Carter Group', symbol='CAROUP', open=113.16, low=109.41, high=116.91, close=109.41, weight=185),
 Company(name='Lang Inc', symbol='LANINC', open=32.37, low=33.41, high=31.33, close=33.41, weight=93),
 Company(name='Price-Howard', symbol='PRIARD', open=369.64, low=394.58, high=344.7, close=394.58, weight=100),
 Company(name='Gomez Inc', symbol='GOMINC', open=128.23, low=125.97, high=130.49, close=125.97, weight=102),
 Company(name='Walsh, Anderson and Hill', symbol='WALILL', open=964.31, low=880.97, high=1047.65, close=880.97, weight=166),
 Company(name='Miller Group', symbol='MILOUP', open=775.04, low=736.83, high=813.25, close=736.

In [156]:
stock_list.sort(key = lambda a: a.weight)

In [157]:
stock_market_open = sum([a.open for a in stock_list[:50]])
stock_market_high = sum([a.high for a in stock_list[:50]])
stock_market_close = sum([a.close for a in stock_list[:50]])

In [158]:
print('stock market opened at:',stock_market_open)
print('stock market highest value was:',stock_market_high)
print('stock market closed at:',stock_market_close)

stock market opened at: 22447.309999999998
stock market highest value was: 22367.46
stock market closed at: 22527.160000000003
